In [1]:
# Import modules
import numpy as np
import gc
import matplotlib.pyplot as plt
import keras.utils as image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve
import numpy as np
from keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet

import tensorflow as tf
from tensorflow.keras import layers, models

# Import modules
import tensorflow as tf
import tensorflow.keras as K


from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, BatchNormalization

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
import keras

# import keras.utils as image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

2025-03-10 01:20:31.295110: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-10 01:20:31.305289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741544431.316440  749460 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741544431.319700  749460 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 01:20:31.332929: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
N_MELS = 128

In [ ]:
X = np.load('../SavedFeatures/X_mel_spec.npy')
y = np.load('../SavedFeatures/y_mel_spec.npy')

X_train, X_val, y_train, y_val= train_test_split(X, y, test_size = 0.2, random_state = 42)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

del X, y  
gc.collect()

In [6]:
def model_resnet(input_shape, num_classes):
    base_model = ResNet50(include_top=False, weights=None, input_shape=input_shape)
    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=x)

In [7]:
tf.keras.backend.clear_session()

In [8]:
mrsnt = model_resnet(input_shape=(N_MELS, 109, 1), num_classes=2)
mrsnt.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


I0000 00:00:1741544442.177419  749460 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12015 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9


In [9]:
hist = mrsnt.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10


I0000 00:00:1741544456.745975  749812 service.cc:148] XLA service 0x7602ac0021b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741544456.745992  749812 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Ti, Compute Capability 8.9
2025-03-10 01:20:57.136751: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741544458.770744  749812 cuda_dnn.cc:529] Loaded cuDNN version 90300


  5/637 ━━━━━━━━━━━━━━━━━━━━ 19s 31ms/step - accuracy: 0.5356 - loss: 0.8243 

I0000 00:00:1741544467.281833  749812 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


637/637 ━━━━━━━━━━━━━━━━━━━━ 55s 50ms/step - accuracy: 0.6496 - loss: 0.6860 - val_accuracy: 0.8210 - val_loss: 0.4085
Epoch 2/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.8575 - loss: 0.3738 - val_accuracy: 0.9011 - val_loss: 0.2762
Epoch 3/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.8792 - loss: 0.3109 - val_accuracy: 0.9030 - val_loss: 0.2295
Epoch 4/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.9036 - loss: 0.2459 - val_accuracy: 0.8944 - val_loss: 0.2469
Epoch 5/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.9315 - loss: 0.1773 - val_accuracy: 0.8857 - val_loss: 0.3631
Epoch 6/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.9418 - loss: 0.1508 - val_accuracy: 0.9187 - val_loss: 0.2684
Epoch 7/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.9456 - loss: 0.1433 - val_accuracy: 0.9541 - val_loss: 0.1249
Epoch 8/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.9557 - loss: 0.1276 - val_accurac

In [13]:
test_loss, test_accuracy = mrsnt.evaluate(X_val, y_val, verbose=0)
y_pred = mrsnt.predict(X_val)


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


In [14]:
y_pred_classes = y_pred.argmax(axis=1) 
y_true_classes = y_val.argmax(axis=1) 


In [ ]:
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("===================ResNet50 - MS===================")
print(f"TAccuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")
eers = []

# Tính EER
for i in range(y_pred.shape[1]):  
    
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]

    
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")


mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")


===================ResNet50 - MS===================
TAccuracy: 0.92619
F1-Score: 0.92641
Class 0: EER = 0.07339 at threshold 0.48954
Class 1: EER = 0.07344 at threshold 0.51407
EER: 0.07341


In [16]:
# Save Model as weight with H5 format
mrsnt.save_weights("../SavedModel/MS_mrsnet.weights.h5")

In [ ]:

del mrsnt, X_train, X_val, y_train, y_val
gc.collect()


1793